In [ ]:
import pandas as pd
from zipfile import ZipFile
import pyarrow.parquet as pq
import gzip
import json
import pyarrow as pa
import os

In [ ]:
COLAB = False

In [ ]:
import tempfile

DATA_DIR = "gdrive/MyDrive/Work/qjn/" if COLAB else "data/"

FULL_TEXT_DEV_PATH = DATA_DIR + "newsarticles_article_dev.parquet"

tmp_jsonl = tempfile.NamedTemporaryFile()

PRED_BIN__PATH = DATA_DIR + "newsarticles_ner_pred_dev.spacy"

DATA_DIR = "gdrive/MyDrive/Work/quantify-news/data/" if COLAB else "data/"
PROJECT_DIR = "gdrive/MyDrive/Work/quantify-news/" if COLAB else "./"
TMP_DIR = "/content/" if COLAB else "./"

GPU_ID = "0" if COLAB else "-1"

DATA_TRAIN_BIN_PATH = DATA_DIR + "ner_train.spacy"

# For optuna outer loop
BEST_MODEL_OPT_PATH = PROJECT_DIR + "models/model-best/"


In [ ]:
article_data = pd.read_parquet(FULL_TEXT_DEV_PATH)

In [ ]:
article_data['bodytext'] = (article_data['bodytext']
                                  .str.replace('\n',' ')
                                  .str.replace(u'\xa0', u' '))
    

In [ ]:
article_data = article_data[article_data.relevant]

In [ ]:
import spacy
from spacy import displacy
from spacy.tokens import DocBin
from itertools import batched
nlp = spacy.load(BEST_MODEL_OPT_PATH)

In [24]:
for batch in batched(article_data['bodytext'], 64):
    docs = nlp.pipe(batch)
    docs = list(filter(lambda d: any(map(lambda e: e.label_ == 'NEWS_LOC', d.ents)), docs))
    # sentence_spans = [list(doc.sents) for doc in docs]
    if docs:
        displacy.render(docs, style="ent")
        break

KeyboardInterrupt: 